# 1. Introdução

Neste notebook desenvolvemos um **Modelo de Recomendação de Ativos** utilizando **Filtragem Colaborativa** para sugerir investimentos a clientes que estejam *fora de conformidade* com o Perfil de Carteira escolhido no momento da abertura da conta no **BTG**.

O fluxo completo contempla:
- Carregamento e inspeção dos dados
- Classificação de cada ativo em **Renda Fixa (RF)**, **Renda Variável (RV)** ou **Outros**
- Cálculo de *compliance* das carteiras em relação às regras de perfil
- Seleção da base de usuários **em conformidade** para treinar o modelo
- Construção da matriz usuário‑ativo e treinamento do algoritmo de Filtragem Colaborativa
- Geração de um **ranking de ativos** personalizados para usuários *não* conformes

Siga as seções numeradas (1, 1.1, 1.1.1, …) para um entendimento hierárquico.

## 1.1 Bibliotecas e Configurações

In [1]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import scipy.sparse as sp
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


## 1.2 Carregamento dos Dados

In [2]:

# Ajuste o caminho caso necessário
df = pd.read_csv('carteiras_btg.csv')
print(f"Shape: {df.shape}")
df.head()


Shape: (286145, 6)


,Conta,Financeiro,Quantidade,Nome_Ativo,Tipo_de_Ativo,Perfil_da_carteira
0,27,25081.618073,25.0,CRA CDIE,TITULOS RF PRIVADOS BRASIL,Moderado
1,27,47046.248881,49.0,CRA IPCA,TITULOS RF PRIVADOS BRASIL,Moderado
2,27,36331.476685,35.0,RRRP13 IPCA,DEBENTURES BRASIL,Moderado
3,27,18995.623640,20.0,EEELA1 IPCA,DEBENTURES BRASIL,Moderado
4,27,39630.685146,39.0,TRRG12 IPCA,DEBENTURES BRASIL,Moderado


## 1.3 Dicionário de Classificação dos Ativos


A seguir construímos manualmente um dicionário `map_tipo_para_classe` que mapeia os valores da coluna **`Tipo_de_Ativo`** para uma das três macro‑classes exigidas:

- **RF** – Renda Fixa  
- **RV** – Renda Variável  
- **OUTROS** – Demais classes (multimercado, cambial, previdência, etc.)

> **Observação 1:** O dicionário foi elaborado a partir dos tipos realmente presentes no arquivo CSV.  
> **Observação 2:** Caso surjam novos tipos no futuro, basta adicioná‑los ao dicionário.


In [3]:

# Constrói dicionário automaticamente a partir de heurísticas
unique_tipos = df['Tipo_de_Ativo'].unique()

map_tipo_para_classe = {}

rf_keywords = ['RF', 'CDB', 'LCI', 'LCA', 'CRI', 'CRA', 'DEBENTURE', 'TESOURO', 'BOND', 'DÍVIDA', 'CRI', 'CRA']
rv_keywords = ['ACAO', 'AÇÃO', 'ETF', 'FII', 'BDR', 'REIT', 'FIDC', 'FIAGRO', 'STOCK', 'EQUITY', 'RV']
for t in unique_tipos:
    upper = t.upper()
    if any(k in upper for k in rf_keywords):
        map_tipo_para_classe[t] = 'RF'
    elif any(k in upper for k in rv_keywords):
        map_tipo_para_classe[t] = 'RV'
    else:
        map_tipo_para_classe[t] = 'OUTROS'

# Mostra mapeamento resumido
pd.Series(map_tipo_para_classe).value_counts()


OUTROS    61
RF         5
RV         2
Name: count, dtype: int64

## 1.4 Aplicar Classificação aos Dados

In [4]:

df['Classe_Ativo'] = df['Tipo_de_Ativo'].map(map_tipo_para_classe)
df.head()


,Conta,Financeiro,Quantidade,Nome_Ativo,Tipo_de_Ativo,Perfil_da_carteira,Classe_Ativo
0,27,25081.618073,25.0,CRA CDIE,TITULOS RF PRIVADOS BRASIL,Moderado,RF
1,27,47046.248881,49.0,CRA IPCA,TITULOS RF PRIVADOS BRASIL,Moderado,RF
2,27,36331.476685,35.0,RRRP13 IPCA,DEBENTURES BRASIL,Moderado,RF
3,27,18995.623640,20.0,EEELA1 IPCA,DEBENTURES BRASIL,Moderado,RF
4,27,39630.685146,39.0,TRRG12 IPCA,DEBENTURES BRASIL,Moderado,RF


# 2. Conformidade das Carteiras


Para cada **Conta** calculamos a distribuição percentual entre RF, RV e OUTROS.
Em seguida verificamos se a distribuição respeita a regra do **Perfil_da_carteira**:

| Perfil | Regra |
|--------|-------|
| Conservador | ≥ 90 % RF e ≤ 10 % demais |
| Moderado | ≥ 60 % RF e ≤ 40 % demais |
| Sofisticado/Arrojado | ≥ 70 % RV e ≤ 30 % demais |

Criamos a coluna `em_conformidade` (boolean).  


In [5]:

# Percentual por conta & classe
portfolio_pct = (
    df.pivot_table(index='Conta', columns='Classe_Ativo', values='Financeiro', aggfunc='sum')
      .fillna(0)
)
portfolio_pct = portfolio_pct.div(portfolio_pct.sum(axis=1), axis=0)*100
portfolio_pct.head()

# Junta perfil
perfil = df.groupby('Conta')['Perfil_da_carteira'].first()
portfolio_pct = portfolio_pct.join(perfil)

def verifica(row):
    perfil = row['Perfil_da_carteira']
    rf = row.get('RF', 0)
    rv = row.get('RV', 0)
    outros = row.get('OUTROS', 0)
    if perfil == 'Conservador':
        return (rf >= 90) and (rv + outros <= 10)
    elif perfil == 'Moderado':
        return (rf >= 60) and (rv + outros <= 40)
    elif perfil in ['Sofisticado', 'Arrojado', 'Sofisticado/Arrojado']:
        return (rv >= 70) and (rf + outros <= 30)
    else:
        return False

portfolio_pct['em_conformidade'] = portfolio_pct.apply(verifica, axis=1)
portfolio_pct['em_conformidade'].value_counts()


em_conformidade
False    3142
True     2960
Name: count, dtype: int64

# 3. Construção da Matriz Usuário‑Ativo


Usaremos apenas usuários **em conformidade** como *base confiável* para inferir preferências.
A matriz `(users × ativos)` será ponderada pelo **percentual financeiro** do ativo na carteira do usuário.

Para evitar sparsidade excessiva, manteremos apenas ativos presentes em pelo menos **N** carteiras; o valor padrão é `N = 5`.


In [6]:

# Filtra usuários em conformidade
df_ok = df[df['Conta'].isin(portfolio_pct[portfolio_pct['em_conformidade']].index)]

# Financeiro por conta e ativo
user_item = (
    df_ok.pivot_table(index='Conta', columns='Nome_Ativo', values='Financeiro', aggfunc='sum')
       .fillna(0)
)

# Converte valores absolutos em percentuais
user_item = user_item.div(user_item.sum(axis=1), axis=0)

# Remove ativos raros
min_users = 5
item_counts = (user_item > 0).sum(axis=0)
user_item = user_item.loc[:, item_counts >= min_users]

print(user_item.shape)


(2960, 1032)


# 4. Treinamento do Modelo de Filtragem Colaborativa


Usaremos um **KNN baseado em similaridade de itens**:
1. A matriz é convertida para formato esparso `csr_matrix`.
2. A similaridade é calculada via **cosseno**.
3. Para cada ativo, guardamos os *K* vizinhos mais similares (default `K = 20`).

> O modelo é simples, interpretável e dispensa treinamento pesado.


In [7]:

from sklearn.metrics.pairwise import cosine_similarity

K = 20

# sparse matrix
item_user_mat = sp.csr_matrix(user_item.T.values)

# similaridade item×item
sim = cosine_similarity(item_user_mat)

# Mantém apenas top‑K por coluna
topk = np.argsort(-sim, axis=1)[:, 1:K+1]  # ignora self (pos 0)

# Dicionário: item -> vizinhos
item_idx_to_name = dict(enumerate(user_item.columns))
item_neighbors = {
    item_idx_to_name[i]: [item_idx_to_name[j] for j in topk[i]]
    for i in range(sim.shape[0])
}
# exemplo
list(item_neighbors.items())[:3]


[('AALM12 CDIE',
  ['INEL12 CDIE',
   'ALTF11 CDIE',
   'BKBR19 CDIE',
   'HAPV15 CDIE',
   'SVEA22 CDIE',
   'OPCT15 CDIE',
   'MILS18 CDIE',
   'ARML14 CDIE',
   'DASAA6 CDIE',
   'BVPL11 CDIE',
   'TRES11 CDIE',
   'HBSA22 CDIE',
   'TEPA12 IPCA',
   'VAMO24 CDIE',
   'AEGPA9 CDIE',
   'ELTN16 CDIE',
   'CSANB0 CDIE',
   'JSLGA5 CDIE',
   'MATD12 CDIE',
   'MVLV19 CDIE']),
 ('AALR13 CDIE',
  ['SLZG12 IPCA',
   'RAIN12 IPCA',
   'CGCE12 IPCA',
   'BGCE12 IPCA',
   'SVEA22 CDIE',
   'ASAB11 IPCA',
   'CSMGB9 IPCA',
   'DASAC5 CDIE',
   'GRRB24 IPCA',
   'ONCO19 CDIE',
   'VLME11 CDIE',
   'CNRD11 IPCA',
   'CAEC21 IPCA',
   'ELFA12 CDIE',
   'VAMO34 IPCA',
   'VERT15 CDIE',
   'NTSD17 CDIE',
   'TEPA13 IPCA',
   'DESK15 CDIE',
   'CASN13 CDIE']),
 ('AAZQ11',
  ['OMGE22 IPCA',
   'CPLD26 IPCA',
   'CSNAC4 IPCA',
   'RISP22 IPCA',
   'CGOS13 IPCA',
   'CDCA PRE',
   'ELET16 IPCA',
   'VERO14 CDIE',
   'RIS422 IPCA',
   'CRA PRE',
   'VERO34 PRE',
   'TAEED2 IPCA',
   'CRI PRE',
   'TAEE

# 5. Geração de Ranking Personalizado


Para cada **usuário fora de conformidade**:
1. Identificamos os ativos já possuídos.
2. Agregamos as similaridades dos vizinhos dos ativos possuídos (excluindo os já presentes).
3. Ordenamos pelo score acumulado para obter o ranking.

A função `recomendar_ativos` abaixo retorna o *top‑N* (default `N = 10`) para qualquer **`id_conta`**.


In [8]:

# users out‑of‑compliance
usuarios_alvo = portfolio_pct[~portfolio_pct['em_conformidade']].index.tolist()

def recomendar_ativos(id_conta, N=10):
    if id_conta not in usuarios_alvo:
        raise ValueError("Usuário não está fora de conformidade ou não existe.")
    possuidos = set(df[df['Conta'] == id_conta]['Nome_Ativo'])
    scores = {}
    for ativo in possuidos:
        vizinhos = item_neighbors.get(ativo, [])
        for v in vizinhos:
            if v in possuidos:
                continue
            scores[v] = scores.get(v, 0) + 1  # soma votos
    return sorted(scores.items(), key=lambda x: -x[1])[:N]

# Exemplo de uso
test_user = usuarios_alvo[0]
recomendar_ativos(test_user, N=10)


[('MSGT23 IPCA', 31),
 ('GASC25 IPCA', 27),
 ('ELTE12 IPCA', 14),
 ('POTE12 IPCA', 13),
 ('ENEV15 IPCA', 11),
 ('RISP22 IPCA', 11),
 ('MRSAB1 IPCA', 10),
 ('GASC17 IPCA', 6),
 ('ELTN15 IPCA', 5),
 ('CPLD29 IPCA', 5)]

# 6. Avaliação Rápida


Como não possuímos *ground‑truth* de compras futuras, realizamos uma avaliação **offline** simples:

- **Cobertura (@K)** – proporção de usuários para os quais o modelo consegue indicar pelo menos 1 ativo novo.
- **Diversidade** – nº médio de ativos distintos sugeridos em toda a base de teste.

Isso dá um termômetro inicial de utilidade do ranking.


In [9]:

def cobertura(k=10):
    hit = 0
    total = len(usuarios_alvo)
    all_rec = set()
    for u in usuarios_alvo:
        recs = recomendar_ativos(u, N=k)
        if recs:
            hit += 1
            all_rec.update([r[0] for r in recs])
    return hit/total, len(all_rec)

cov, div = cobertura()
print(f"Cobertura: {cov:.2%}")
print(f"Diversidade (#ativos recomendados): {div}")


Cobertura: 94.18%
Diversidade (#ativos recomendados): 729


## 6.1 Protocolo de Avaliação Offline (Hold-Out)

Para medir personalização sem dados futuros de compra, usamos um **leave-one-out** por usuário *em conformidade*:

1. Para cada carteira, **remove-se** aleatoriamente **um** ativo (`item_holdout`).
2. Reconstrói-se o modelo **sem** esse ativo.
3. Gera-se Top-K recomendações e verifica-se se `item_holdout` aparece na lista.

Métricas calculadas:

| Métrica | Descrição |
|---------|-----------|
| **Hit Rate @K** | % de usuários cujo item oculto aparece no Top-K |
| **Precision @K** | média de `hits/K` |
| **Recall @K** | média de `hits/1` (só 1 item relevante) |
| **MAP @K** | Mean Average Precision |
| **NDCG @K** | Normalized DCG |


In [10]:
# --- 6.1 Avaliação Offline (FIX) ---
def holdout_evaluation(user_item_full, K=10, seed=42):
    """
    Leave-one-out sobre usuários em conformidade.
    Retorna HitRate, Precision, Recall, MAP e NDCG @K.
    """
    rng = np.random.default_rng(seed)
    hits = prec_sum = dcg_sum = 0
    users_eval = 0
    
    mat = user_item_full.values          # matriz numpy (users × items)
    cols = user_item_full.columns        # nomes de ativos
    
    for u in range(mat.shape[0]):
        row = mat[u]                     # array 1-D
        itens_pos = np.where(row > 0)[0] # índices de itens possuídos
        if len(itens_pos) < 2:           # precisa ter >1 para esconder 1
            continue
        
        users_eval += 1
        hold_idx  = rng.choice(itens_pos)
        hold_item = cols[hold_idx]
        
        possuidos_idx = np.setdiff1d(itens_pos, [hold_idx])
        possuidos = set(cols[possuidos_idx])
        
        # agrega votos de vizinhos
        scores = {}
        for a in possuidos:
            for v in item_neighbors.get(a, []):
                if v in possuidos:       # já possui
                    continue
                scores[v] = scores.get(v, 0) + 1
        
        recs = [x for x, _ in sorted(scores.items(), key=lambda x: -x[1])][:K]
        
        if hold_item in recs:
            hits += 1
            pos = recs.index(hold_item)
            prec_sum += 1/(pos+1)            # AP com 1 relevante
            dcg_sum += 1/np.log2(pos+2)      # DCG
  
    if users_eval == 0:                      # borda: ninguém elegível
        return {}
    
    hr   = hits / users_eval
    prec = hits / (users_eval*K)
    rec  = hits / users_eval                # recall = hit/1
    mapk = prec_sum / users_eval
    ndcg = dcg_sum / users_eval
    
    return {'Users': users_eval,
            'HitRate@K': hr,
            'Precision@K': prec,
            'Recall@K': rec,
            'MAP@K': mapk,
            'NDCG@K': ndcg}

evaluation_results = holdout_evaluation(user_item, K=10)
evaluation_results


{'Users': 2090,
 'HitRate@K': 0.47129186602870815,
 'Precision@K': 0.04712918660287081,
 'Recall@K': 0.47129186602870815,
 'MAP@K': 0.27442583732057385,
 'NDCG@K': np.float64(0.32084093131698477)}

## 6.3 Diversidade e Novidade

* **Intra-lista Diversidade** – 1 − (frequência da classe mais comum / K).  
* **Novidade** – média de $(1/\\log_2(popularidade+2))$ dos ativos recomendados.  
Visam evitar listas redundantes e fomentar descoberta.


In [11]:
# --- 6.3 Diversidade / Novidade ---
def diversity_and_novelty(k=10):
    pop = df_ok['Nome_Ativo'].value_counts()
    divs, novs = [], []
    for u in usuarios_alvo:
        recs = recomendar_ativos(u, N=k)
        if not recs: continue
        classes = [map_tipo_para_classe[
            df[df['Nome_Ativo']==r[0]]['Tipo_de_Ativo'].iloc[0]] for r in recs]
        divs.append(1 - max(classes.count(c) for c in ['RF','RV','OUTROS'])/k)
        novs.append(np.mean([1/np.log2(pop[r[0]]+2) for r in recs]))
    return {'Users':len(divs),'Mean_Diversity':np.mean(divs),'Mean_Novelty':np.mean(novs)}

div_nov_results = diversity_and_novelty(k=10)
div_nov_results


{'Users': 2959,
 'Mean_Diversity': np.float64(0.08844204123014533),
 'Mean_Novelty': np.float64(0.18273405926459044)}

# 8 Exportação do modelo
- Exportaremos o modelo para um arquivo `.pkl`.

In [12]:
## 8.1 Salvar modelo em arquivo .pkl
import pickle
from pathlib import Path

# agrupa tudo que precisamos para reproduzir as recomendações
model_artifact = {
    "K": K,                                           # vizinhos por item
    "item_names": user_item.columns.tolist(),         # ordem usada na matriz
    "similarity_matrix": sim,                         # numpy.ndarray
    "neighbors_dict": item_neighbors                  # já filtrado com top-K
}

output_path = Path("item_knn_model.pkl")
with open(output_path, "wb") as f:
    pickle.dump(model_artifact, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Modelo salvo em: {output_path.resolve()}")


Modelo salvo em: C:\Users\Caleb\GitHub\Modelo-de-Recomendacao_Sprint-2\item_knn_model.pkl
